In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 66kB/s 
     |████████████████████████████████| 204kB 48.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=bf39fc1c0d6a141fd0939a4a4becc983b0da463da585292291a5f4254a5516e4
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('reg_example').getOrCreate()

In [4]:
 from pyspark.ml.regression import LinearRegression

In [6]:
 customer_data = spark.read.csv("customers.csv", inferSchema = True, header = True)

In [7]:
customer_data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [20]:
customer_data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [22]:
customer_data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [23]:
from pyspark.ml.linalg import Vectors

In [25]:
 from pyspark.ml.feature import VectorAssembler

In [26]:
customer_data.columns 

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [27]:
assembler = VectorAssembler(inputCols = ['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol = 'features')

In [29]:
output = assembler.transform(customer_data)

In [31]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [34]:
final_data = output.select('features', 'Yearly Amount Spent')

In [35]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [36]:
train, test = final_data.randomSplit([0.7,0.3]) 

In [37]:
 train.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                333|
|   mean| 498.54577288244036|
| stddev|  78.49702929734634|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [38]:
test.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                167|
|   mean|  500.8459686194174|
| stddev|  81.13746438355273|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [39]:
## Create model with train data set
## And evaluate our model with test data set

In [40]:
lr = LinearRegression(labelCol = "Yearly Amount Spent")

In [42]:
linear_regression_model = lr.fit(train)

In [43]:
results = linear_regression_model.evaluate(test)

In [45]:
results.residuals.show() ## values that were failed

+-------------------+
|          residuals|
+-------------------+
| 11.627431919279957|
| -17.45906716220327|
| -2.905634822978243|
|   8.02921081892623|
| -5.027644968276093|
| 10.197999908161137|
|  23.64216471915256|
|  18.76056397642452|
| 0.4001360304253012|
| -3.285174789563598|
|-7.6201806985519625|
| -5.804317361621372|
|  18.35342821795257|
| -6.666565689394588|
|-2.3884288549758708|
| -4.644163958713136|
|-10.216754026503395|
| -9.547474454194514|
|-1.3916017431587306|
|-6.0017423579378715|
+-------------------+
only showing top 20 rows



In [51]:
results.rootMeanSquaredError ## Average of error in prediction, in our case we're handling are +/- 500, 10 is a low number

10.596113559841717

In [52]:
 results.r2  ## the result gives us that is a good model, +/- 98% accurate prediction

0.9828423135033207